# 📸 [DACON - CNN](https://dacon.io/codeshare/4537)

---

- 영상처리에 탁원한 성능을 자랑하는 **CNN**의 원리를 알아보고, MNIST에 적용해보도록 한다.

- 다음으로, CNN을 이용한 **ResNet** 모델로 좀 더 복잡한 컬러 이미지를 다뤄보도록 한다.<br><br>

- 현재 과정에서는 CNN 원리에 대해서 간단하게 알아보고, 자세한 모델 Architecture은 추후에 다뤄보도록 한다.

- 컴퓨터에서 보는 모든 이미지는 픽셀값들을 가로, 세로로 늘어놓은 행렬로 표현할 수 있다.

- **`컨볼루션`** 은 계층적으로 이미지를 인식할 수 있도록 단계마다 이미지의 특징을 추출해주는 것을 의미한다.

- **CNN**은 이미지를 추출하는 필터로 Convolution Neural Network, **즉 컨볼루션을 하는 인공 신경망이다.**

- CNN 모델은 일반적으로 **`Convolution Layer`**, **`Pooling Layer`**, 특징들을 모아 최종 분류하는 일반적인 인공신경망 계층으로 구성된다.

- 컨볼루션을 거쳐 만들어진 새로운 이미지는 **`특징 맵(Feature Map)`** 이라고도 불린다.

- 컨볼루션 계층마다 여러 특징 맵들이 만들어지며, 다음 단계인 **풀링(Pooling)** 계층으로 넘어가게 된다.

- 컨볼루션 계층과 폴링 계층을 여러 겹 쌓아, 각 단계에서 만들어진 특징 맵을 관찰하면 CNN 모델이 이미지를 계층적으로 인식하는 것을 볼 수 있다.

- **특징 맵의 크기가 크면 학습이 어렵고, 과적합의 위험이 증가한다.**

# <span style="color:brown">01. CNN 모델 구현하기</span>

---

- 여러 CNN 모델은 **Convolution, Pooling, Dropout**, 그리고 **일반적인 신경망 계층**의 조합으로 이루어진다.

- **Convolution → Pooling → Convolution → Dropout → Pooling → Flatten → Fully Connected → Dropout → Fully Connected** 예제를 구현해보도록 한다.

- 일반 인공신경망을 CNN 계층으로 대체하면 되기 때문에 전체적 구현은 DNN 신경망 구현법과 매우 비슷하다.

In [56]:
# 라이브러리 불러오기
import torch

from torch import nn
from torch import optim
from torch.nn import functional as F
from torchvision import transforms, datasets

print("PyTorch Version :", torch.__version__)

PyTorch Version : 1.10.2


In [57]:
torch.manual_seed(1)

In [58]:
USE_CUDA = torch.cuda.is_available()
DEVICE = "cuda" if USE_CUDA else "cpu"

print("Using Device :", DEVICE)

Using Device : cpu


## <span style="color:orange">1. Hyperparameters</span>

---

In [59]:
# 에폭과 배치크기를 정해주도록 한다.
EPOCHS = 40
BATCH_SIZE = 64

## <span style="color:orange"> 2. Data Load</span>

---

In [60]:
# MNIST 데이터 불러오기
train_loader = torch.utils.data.DataLoader(
    
    # 'MNIST()' 함수 적용
    datasets.MNIST(
                   root = "./PyTorch로 시작하는 딥러닝 입문/MNIST data/",
                   train = True,
                   download = False,
                   transform = transforms.Compose([
                       
                       # PyTorch 텐서화와 정규화 수행
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size = BATCH_SIZE,
    shuffle = True)

print("Shape of First Mini Batch in MNIST train :", list(train_loader)[0][0].size(), "\n")
print("Shape of First Mini Batch Target in MNIST train :", list(train_loader)[0][1].size(), "\n")

# 테스트 데이터 불러오기
test_loader = torch.utils.data.DataLoader(

    datasets.MNIST(
                   root = "./PyTorch로 시작하는 딥러닝 입문/MNIST data/",
                   train = False,
                   download = False,
                   transform = transforms.Compose([
                       
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size = BATCH_SIZE,
    shuffle = True)

print("Shape of First Mini Batch in MNIST test :", list(test_loader)[0][0].size(), "\n")
print("Shape of First Mini Batch Target in MNIST test :", list(test_loader)[0][1].size())

Shape of First Mini Batch in MNIST train : torch.Size([64, 1, 28, 28]) 

Shape of First Mini Batch Target in MNIST train : torch.Size([64]) 

Shape of First Mini Batch in MNIST test : torch.Size([64, 1, 28, 28]) 

Shape of First Mini Batch Target in MNIST test : torch.Size([64])


- MNIST 데이터는 각 이미지가 28 $\times$ 28 픽셀로 이루어져 있음을 확인할 수 있으며, 위에서 지정한 Batch Size만큼 DataLoader가 생성되었다.

- 위에서 사용된 `transforms.Compose()` 함수는 여러 transforms의 함수들을 구성해주며, 리스트 객체를 입력으로 넣어주면 된다.

- 사용된 전처리는 **PyTorch 텐서화**와 **정규화**이다.

## <span style="color:orange">3. CNN Model</span>

---

In [61]:
# CNN 모델 클래스 생성
class Net(nn.Module):
    
    def __init__(self):
        
        super(Net, self).__init__()
        
        #-----------------
        # 첫번째 컨볼루션 계층
        #-----------------
        
        # 생성한 모델의 커널 크기는 '5 x 5' 이다. 'kernel_size' 매개변수에 숫자를 지정하면 정사각형으로 간주한다.
        # 첫번째 컨볼루션 계층을 통해 '10개의 특징맵을 생성한다.'
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 10, kernel_size = 5)
        
        #-----------------
        # 두번째 컨볼루션 계층
        #-----------------
        
        # 두번째 컨볼루션 계층에서는 10개의 특징맵을 받아 20개의 특징맵을 반환하도록 한다.
        self.conv2 = nn.Conv2d(in_channels = 10, out_channels = 20, kernel_size = 5)
        
        #-----------------
        # 드롭아웃 계층
        #-----------------
        
        # 컨볼루션 결과 출력값에는 드롭아웃을 해주도록 한다.
        # 'p' 매개변수에 따로 지정한 값이 없다면, 0.5의 비율만큼 드롭아웃을 수행한다.
        self.conv2_drop = nn.Dropout2d()
        
        #-----------------
        # Fully Connected Layer
        #-----------------
        self.fc1 = nn.Linear(320, 50)
        
        # 마지막 출력 결과는 분류할 클래스 개수인 10으로 출력을 설정해주어야 한다.
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        
        # 입력 받은 값이 첫번째 컨볼루션 계층을 거치고 'F.max_pool2d()' 함수를 거친다.
        # Convolution → Pooling 과정을 거친 후, ReLU 활성화 함수를 거친다.
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 2))
        
        # 두번째 컨볼루션 계층은 다음과 같은 순서로 진행된다.
        # Convolution → Dropout → Pooling → Activation Function
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), kernel_size = 2))
        
        # 'Fully Connected Layer'에 적용하기 전에 특징맵이 된 x를 1차원으로 Flatten 해주도록 한다.
        x = x.view(-1, 320)
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        
        return x

- 위에서 **CNN Architecture**를 아래와 같이 구성하였다.

- **`Convolution → Pooling → Convolution → Dropout → Pooling → Flatten → Fully Connected → Dropout → Fully Connected`**

- 바로 위 `Net` 클래스의 코드를 하나씩 뜯어보면서 살펴보도록 한다. (텐서의 형태 변화를 보고자 하는 것이기 때문에 활성화 함수는 생략하도록 한다.)

- 우선 `train_loader`에 저장되어 있는 형태와 똑같은 텐서를 생성해주도록 한다.

In [62]:
# 임의의 텐서 생성
tensor = torch.randn(64, 1, 28, 28)
print("Shape of Sample Tensor :", tensor.size())

Shape of Sample Tensor : torch.Size([64, 1, 28, 28])


1️⃣ **`Convolution`**

- 첫번째 Convolution 계층을 생성하고 적용한 후, 어떻게 변화하는지 살펴보도록 한다.

In [63]:
# 첫번째 Convolution 계층
conv1 = nn.Conv2d(in_channels = 1, out_channels = 10, kernel_size = 5)

# 첫번째 Convolution 계층을 거친 후 확인
x = conv1(tensor)
print("Shape of Applied First Convolution Layer :", x.size())

Shape of Applied First Convolution Layer : torch.Size([64, 10, 24, 24])


- `nn.Conv2d()` 함수를 지나면 형태는 $(N, C_{in}, H, W) \to (N, C_{out}, H_{out}, W_{out})$과 같이 변하게 된다.

- 즉 현재 입력값으로 받은 이미지의 channels는 1이고, `out_channels = 10`으로 지정되어 있으므로 10개의 특징맵이 생성된 것을 확인할 수 있다.

- 또한, `kernel_size = 5` 이므로 28 x 28 픽셀의 이미지가 24 x 24 픽셀로 줄어들었음을 확인할 수 있다.

2️⃣ **`Convolution → Pooling`**

- Convolution 계층 다음으로는 Pooling 층을 적용해보도록 한다.

- Pooling 층을 적용할 때, `torch.nn.functional.max_pool2d()` 함수 혹은 `torch.nn.MaxPool2d()` 함수를 사용해도 된다.

- 지금 과정에서는 `torch.nn.MaxPool2d()` 함수를 사용해보도록 한다.

In [64]:
# 첫번째 Pooling 계층
pool1 = nn.MaxPool2d(kernel_size = 2)

# 적용 후 확인
x = pool1(x)
print("Shape of Applied First Pooling Layer :", x.size())

Shape of Applied First Pooling Layer : torch.Size([64, 10, 12, 12])


- 첫번째 Pooling 계층에 `kerner_size = 2` 이므로 24 x 24 픽셀의 이미지가 절반인 12 x 12 픽셀로 줄어들었음을 확인할 수 있다.

3️⃣ **`Convolution → Pooling → Convolution`**

- 이제 두번째 Convolution 계층을 생성해보도록 한다.

- 첫번째 Convolution 계층을 통해 10개의 특징맵이 생성되었고, 10개의 특징맵을 입력으로 받아 20개의 특징맵을 반환하는 계층을 생성해보도록 한다.

In [65]:
# 두번째 Convolution 계층
conv2 = nn.Conv2d(in_channels = 10, out_channels = 20, kernel_size = 5)

# 두번째 Convolution 계층 거친 후 확인
x = conv2(x)
print("Shape of Applied Second Convolution Layer :", x.size())

Shape of Applied Second Convolution Layer : torch.Size([64, 20, 8, 8])


- 두번째 Convolution 계층을 지나고 난 후, 다음과 같이 형태가 변형되었다.

- `torch.Size([64, 10, 12, 12]) → torch.Size([64, 20, 8, 8])`

4️⃣ **`Convolution → Pooling → Convolution → Dropout → Pooling`**

- 두번째 Convolution 계층을 지나고 나면, Dropout과 Pooling 계층을 지나면 된다.

- Dropout 계층을 통해서는 형태 변환이 없기 때문에 Pooling 계층까지 한번에 진행하도록 한다.

- 두번째 Pooling 계층에서도 마찬가지로 `kernel_size = 2`로 지정해주도록 한다.

In [66]:
# 1. Dropout 계층 생성
drop1 = nn.Dropout2d()

# Dropout 계층 적용
x = drop1(x)
print("Shape of Applied First Dropout Layer :", x.size(), "\n")

# 2. Pooling 계층 생성
pool2 = nn.MaxPool2d(kernel_size = 2)

# Pooling 계층 적용
x = pool2(x)
print("Shape of Applied Second Pooling Layer :", x.size())

Shape of Applied First Dropout Layer : torch.Size([64, 20, 8, 8]) 

Shape of Applied Second Pooling Layer : torch.Size([64, 20, 4, 4])


- Dropout 계층을 통해서는 형태 변환이 없고, Pooling 계층을 지나고 난 후 형태가 변했음을 확인할 수 있다.

- `kernel_size = 2` 이므로 8 x 8 픽셀의 이미지가 절반인 4 x 4 픽셀로 줄어들었다.

5️⃣ **`Convolution → Pooling → Convolution → Dropout → Pooling → Flatten`**

- Convolution, Pooling, Dropout을 통해 **Feature Extraction** 과정을 진행했으므로 그 다음으로는 **Fully Connected Layer**를 통해 이미지를 분류해주면 된다.

- Fully Connected Layer에 입력값을 넣어주기 전에 이미지 데이터이므로 입력값을 1차원 벡터로 변경해주는 작업이 필요하다.

- 지금까지의 과정을 거쳐 20개의 특성맵과 4 x 4 픽셀의 형태를 가지기 때문에, $20 \times 4 \times 4 = 320$ 이다.

- 즉, 320개의 특성을 가지는 1차원 벡터를 생성해주면 된다.

In [67]:
# Flatten 과정 진행
x = x.view(-1, 320)
print("Shape of Flatten tensor :", x.size())

Shape of Flatten tensor : torch.Size([64, 320])


6️⃣ **`Convolution → Pooling → Convolution → Dropout → Pooling → Flatten → Fully Connected`**

- 2차원 이미지 데이터의 Flatten 과정을 거쳐 1차원 벡터로 변경해주었으므로, `torch.nn.Linear()` 함수를 통해 Fully Connected Layer를 생성하면 된다.

- 320개의 입력값을 받아서 50개를 출력값으로 반환하는 계층을 생성하도록 한다.

In [68]:
# 첫번째 Fully Connected 계층 생성
fully1 = nn.Linear(320, 50)

# Fully Connected 계층 적용
x = fully1(x)
print("Shape of Applied First Fully Connected Layer :", x.size())

Shape of Applied First Fully Connected Layer : torch.Size([64, 50])


7️⃣ **`Convolution → Pooling → Convolution → Dropout → Pooling → Flatten → Fully Connected → Dropout → Fully Connected`**

- 이제 마지막으로 Dropout과 Fully Connected 계층을 생성해주고 적용해주도록 한다.

- 또한, 마지막 Fully Connected 계층에는 출력값이 해당 데이터 클래스의 개수이어야 한다.

- 현재 과정에서는 10이다.

In [69]:
# 1. Dropout 계층 생성
drop2 = nn.Dropout()

# Dropout 계층 적용
x = drop2(x)
print("Shape of Applied Second Dropout Layer :", x.size(), "\n")

# 2. Fully Connected 계층 생성
fully2 = nn.Linear(50, 10)

# Fully Connected 계층 적용
x = fully2(x)
print("Shape of Applied Second Fully Connected Layer :", x.size())

Shape of Applied Second Dropout Layer : torch.Size([64, 50]) 

Shape of Applied Second Fully Connected Layer : torch.Size([64, 10])


- 이로써 7번의 과정을 통해 위에서 정의한 CNN 모델 클래스 객체를 살펴보았다.

- 7번 과정에서 주의할 점은 Dropout을 진행할 때 **`torch.nn.Dropout()`** 함수를 사용하였고, 4번 과정에서는 **`torch.nn.Dropout2d()`** 함수를 사용하였다는 것이다.

- 4번 과정에서는 2차원의 이미지 데이터이기 때문에 `torch.nn.Dropout2d()`를 사용하였고, 7번 과정에서는 Flatten을 진행해준 1차원 데이터이기 때문에 `torch.nn.Dropout()`를 사용하였다.

## <span style="color:orange">4. Model Train</span>

---

In [79]:
# 모델 객체 생성
model = Net().to(DEVICE)
print(model)

# Optimizer 생성
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [80]:
# 훈련을 진행할 수 있는 함수 생성
def train(model, train_loader, optimizer, epoch):
    
    # 모델을 학습 모드로 전환
    model.train()
    
    # 반복문을 통해 학습 진행
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(DEVICE), target.to(DEVICE)
        
        # 입력된 모델을 통해 데이터 학습
        output = model(data)
        
        # Gradients 초기화
        optimizer.zero_grad()
        
        # 분류 문제이기 때문에 'torch.nn.functional.cross_entropy()' 함수를 사용한다.
        # 'torch.nn.functional.cross_entropy()' 함수는 소프트맥스 함수까지 포함하고 있음을 기억해야 한다.
        loss = F.cross_entropy(output, target)
        
        # Gradient Descent 수행
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print("Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(epoch,
                                                                              EPOCHS,
                                                                              batch_idx * len(data),
                                                                              len(train_loader.dataset),
                                                                              100. * batch_idx / len(train_loader),
                                                                              loss.item()))

In [81]:
# 훈련된 모델로 성능을 확인할 수 있는 함수 생성
def evaluate(model, test_loader):
    
    # 훈련된 모델을 평가 모드로 전환
    model.eval()
    
    # 미니 배치를 다 수행한 후, 결과를 확인하기 위해 초기값 지정
    test_loss = 0
    correct = 0
    
    # 가중치 변환이 일어나지 않도록 지정
    with torch.no_grad():
        
        for data, target in test_loader:
            
            data, target = data.to(DEVICE), target.to(DEVICE)
            
            # 학습된 모델을 통해 예측값 생성
            output = model(data)
            
            # 배치의 오차를 합하기
            test_loss += F.cross_entropy(output, target, reduction = "sum").item()
            
            # 소프트맥스 함수를 통해 10개 클래스의 확률이 반환되며, 가장 높은 값을 가진 인덱스가 예측값이다.
            pred = output.max(1, keepdim = True)[1]
            
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    # 위 과정을 통해 모든 배치에 대한 평가가 완료되었으며, 손실값을 모두 더해주었기 때문에 테스트 데이터의 개수(10,000)로 나눠주면 된다.
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

- `evaluate` 함수를 통해 훈련된 모델로 테스트 데이터를 예측하면서 성능을 확인할 수 있다.

- `evaluate` 함수 내부의 코드를 자세히 살펴보도록 한다.

```python
1️⃣ test_loss += F.cross_entropy(output, target, reduction = "sum").item()
```

- 위의 `torch.nn.functional.cross_entropy()` 함수는 Cross Entropy를 구해주며, `reduction = "sum"`은 구해진 손실값을 모두 더하라는 의미이다.

- **그렇다면, 미니 배치를 학습하면서 손실값을 더해주는 이유는 무엇일까?**

현재 과정에서 배치 크기는 64이며, 64개 데이터에 대한 손실값을 $64Loss$ 라고 가정해보도록 하자.

우리가 구하고자 하는 테스트 데이터의 손실값은 10,000개 데이터를 모두 예측한 후의 Cross Entropy 값이다.

즉, 수식으로 보면 아래와 같다. 

$$Cost(W) = \frac{10,000Loss(=\ 64Loss + 64Loss + ... + 64Loss)}{10,000}$$

만약, 위의 `reduction` 매개변수에 `mean` 인자를 넣어주면 아래와 같이 된다.

$$\frac{64Loss}{64} +\ \frac{64Loss}{64} +\ ... +\ \frac{64Loss}{64} \ne \frac{64Loss +\ 64Loss +\ ... +\ 64Loss}{10,000}$$

따라서, 위와 같은 이유 떄문에 미니 배치를 모두 학습한 손실값을 확인하려면 **`reduction = "sum"`** 으로 입력해주어야 한다.

```python
2️⃣ pred = output.max(1, keepdim = True)[1]
```

- 위에서 생성한 CNN 모델을 통해 나온 텐서의 형태는 **`torch.Size([64, 10])`** 이다.

- 64개 각각 관측치가 10개 클래스에 속할 확률을 의미하는 것이다.

- 첫번째 인자 `1`은 행방향으로 연산을 수행하라는 의미이며, `keepdim = True`는 `output` 텐서와 크기를 똑같이 반환하라는 의미이다.

- `keepdim = True`의 경우에는 **torch.Size([64, 1])** 이다.

- `keepdim = False`의 경우에는 **torch.Size([64])** 이다.

- 아래의 코드를 통해 한번 더 확인하도록 한다.

In [82]:
print("CNN Model Output Size :", x.size(), "\n")

# 각 관측치가 10개 클래스에 속할 확률 중, 가장 큰 값 반환
print("Maximum *keepdim = True* :", x.max(1, keepdim = True)[1].size(), "\n")
print("Maximum *Keepdim = False* :", x.max(1)[1].size())

CNN Model Output Size : torch.Size([64, 10]) 

Maximum *keepdim = True* : torch.Size([64, 1]) 

Maximum *Keepdim = False* : torch.Size([64])


```python
3️⃣ correct += pred.eq(target.view_as(pred)).sum().item()
```

- 우선 가장 내부의 `target.view_as(pred)`의 **`view_as()`** 함수는 인자로 주어진 텐서와 똑같은 형태로 만들라는 것이다.

- 현재 `pred` 변수의 형태는 `torch.Size([64, 1])` 이며, `target` 변수의 형태는 `torch.Size([64])` 이다.<br><br>

- 다음으로 `tensor.eq()` 메서드는 요소별로 동등 여부를 비교한다. 예시 결과는 해당 [링크](https://pytorch.org/docs/stable/generated/torch.eq.html#torch.eq)를 통해 확인한다.

- 즉, 예측값과 실제값이 같으면 **True**를 반환하고, 예측값과 실제값이 다르면 **False**를 반환한다.<br><br>

- 위의 과정을 마친 후, 해당 텐서는 True 혹은 False로 이루어져 있으며 `tensor.sum()` 메서드를 사용하면 True 개수의 합을 구해준다.

- 손실값과 마찬가지로 모두 더해준 후, 모든 미니 배치 학습을 마치고 테스트 데이터의 개수로 나눠주면 정확도가 나온다

In [83]:
# Epochs를 반복하면서 진행
for epoch in range(1, EPOCHS + 1):
    
    # 모델 학습 진행
    train(model, train_loader, optimizer, epoch)
    
    # Epoch 반복하면서 테스트 데이터 평가
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print()
    print("[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%\n".format(epoch, test_loss, test_accuracy))

Train Epoch: 1/40 [0/60000 (0%)]	Loss: 2.356333
Train Epoch: 1/40 [12800/60000 (21%)]	Loss: 1.287697
Train Epoch: 1/40 [25600/60000 (43%)]	Loss: 0.771791
Train Epoch: 1/40 [38400/60000 (64%)]	Loss: 0.683238
Train Epoch: 1/40 [51200/60000 (85%)]	Loss: 0.381381

[1] Test Loss: 0.1933, Accuracy: 94.42%

Train Epoch: 2/40 [0/60000 (0%)]	Loss: 0.473227
Train Epoch: 2/40 [12800/60000 (21%)]	Loss: 0.658656
Train Epoch: 2/40 [25600/60000 (43%)]	Loss: 0.372008
Train Epoch: 2/40 [38400/60000 (64%)]	Loss: 0.301509
Train Epoch: 2/40 [51200/60000 (85%)]	Loss: 0.213536

[2] Test Loss: 0.1181, Accuracy: 96.31%

Train Epoch: 3/40 [0/60000 (0%)]	Loss: 0.510756
Train Epoch: 3/40 [12800/60000 (21%)]	Loss: 0.403681
Train Epoch: 3/40 [25600/60000 (43%)]	Loss: 0.301711
Train Epoch: 3/40 [38400/60000 (64%)]	Loss: 0.244048
Train Epoch: 3/40 [51200/60000 (85%)]	Loss: 0.186706

[3] Test Loss: 0.0910, Accuracy: 97.09%

Train Epoch: 4/40 [0/60000 (0%)]	Loss: 0.531123
Train Epoch: 4/40 [12800/60000 (21%)]	Loss: 0.